In [1]:
import numpy as np
import plotly.graph_objects as go
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
from policyengine_core.charts import *

In [38]:
reform_ref = Reform.from_dict({
  "gov.irs.credits.ctc.amount.base[0].amount": {
    "2024-01-01.2100-12-31": 5000
  },
  "gov.irs.credits.ctc.phase_out.amount": {
    "2024-01-01.2100-12-31": 0
  },
  "gov.irs.credits.ctc.refundable.fully_refundable": {
    "2024-01-01.2100-12-31": True
  }
}, country_id="us")

reform_non_ref = Reform.from_dict({
  "gov.irs.credits.ctc.amount.base[0].amount": {
    "2024-01-01.2100-12-31": 5000
  },
  "gov.irs.credits.ctc.phase_out.amount": {
    "2024-01-01.2100-12-31": 0
  }
}, country_id="us")

In [39]:
reform_eliminate_ctc = Reform.from_dict({
  "gov.irs.credits.ctc.amount.adult_dependent": {
    "2024-01-01.2025-12-31": 0
  },
  "gov.irs.credits.ctc.amount.base[0].amount": {
    "2024-01-01.2035-12-31": 0
  },
  "gov.irs.credits.ctc.refundable.individual_max": {
    "2024-01-01.2035-12-31": 0
  }
}, country_id="us")

In [40]:
YEAR = "2025"
DEFAULT_AGE = 40
MAX_INCOME = 500000

def create_situation(filing_status, state_code):
    situation = {
        "people": {
            "adult": {
                "age": {YEAR: DEFAULT_AGE},
            },
            "child": {
                "age": {YEAR: 10},
            }
        },
        "families": {"family": {"members": ["adult", "child"]}},
        "marital_units": {"marital_unit": {"members": ["adult"]}},
        "tax_units": {"tax_unit": {"members": ["adult", "child"]}},
        "households": {
            "household": {"members": ["adult", "child"], "state_name": {YEAR: state_code}}
        },
        "axes": [[
            {
                "name": "employment_income",
                "min": 0,
                "max": MAX_INCOME,
                "count": 501,  # Increased to maintain $1000 intervals
                "period": YEAR,
            }
        ]]
    }
    
    if filing_status == "married":
        situation["people"]["spouse"] = {"age": {YEAR: DEFAULT_AGE}}
        for unit in ["families", "marital_units", "tax_units", "households"]:
            situation[unit][list(situation[unit].keys())[0]]["members"].append("spouse")
        
    return situation

In [41]:
def calculate_income(situation, reform=None):
    simulation = Simulation(situation=situation, reform=reform)
    return simulation.calculate("household_net_income", YEAR)


In [42]:
def create_unified_income_comparison_graph(state_code, scenario='original'):
    colors = {
        "dark_blue": "#18375f",
        "light_blue": "#2976fe",
        "gray": "#c5c5c5",
        "dark_gray": "#5d5d5d"
    }

    married_baseline = calculate_income(create_situation("married", state_code))
    hoh_baseline = calculate_income(create_situation("hoh", state_code))

    x = np.linspace(0, MAX_INCOME, 501)
    fig = go.Figure()

    if scenario == 'original':
        married_non_ref = calculate_income(create_situation("married", state_code), reform_non_ref)
        married_ref = calculate_income(create_situation("married", state_code), reform_ref)
        hoh_non_ref = calculate_income(create_situation("hoh", state_code), reform_non_ref)
        hoh_ref = calculate_income(create_situation("hoh", state_code), reform_ref)

        fig.add_trace(go.Scatter(x=x, y=married_non_ref - married_baseline, mode='lines', name='Married, Current Refundability', line=dict(color=colors['dark_blue'])))
        fig.add_trace(go.Scatter(x=x, y=married_ref - married_baseline, mode='lines', name='Married, Full Refundability', line=dict(color=colors['light_blue'], dash='dash')))
        fig.add_trace(go.Scatter(x=x, y=hoh_non_ref - hoh_baseline, mode='lines', name='Single, Current Refundability', line=dict(color=colors['dark_gray'])))
        fig.add_trace(go.Scatter(x=x, y=hoh_ref - hoh_baseline, mode='lines', name='Single, Full Refundability', line=dict(color=colors['gray'], dash='dot')))
        
        title = 'Household with One Child under JD Vance\'s $5,000 Child Tax Credit Expansion'

    elif scenario == 'ctc_elimination':
        married_eliminate = calculate_income(create_situation("married", state_code), reform_eliminate_ctc)
        hoh_eliminate = calculate_income(create_situation("hoh", state_code), reform_eliminate_ctc)

        fig.add_trace(go.Scatter(x=x, y=married_baseline - married_eliminate, mode='lines', name='Married', line=dict(color=colors['dark_blue'])))
        fig.add_trace(go.Scatter(x=x, y=hoh_baseline - hoh_eliminate, mode='lines', name='Single', line=dict(color=colors['gray'])))
        
        title = 'Impact of the Child Tax Credit for a Household with One Child (2025)'

    fig.update_layout(
        title=title,
        xaxis_title="Earnings",
        yaxis_title="Net Impact",
        xaxis=dict(tickformat='$,.0f', range=[0, MAX_INCOME]),
        yaxis=dict(tickformat='$,.0f'),
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=1.01
        ),
        height=600,
        width=800,
    )

    return fig

In [43]:
# Create and display the original chart
state_code = "TX"
fig_original = create_unified_income_comparison_graph(state_code, scenario='original')
fig_original = format_fig(fig_original)
fig_original.show()

# Create and display the CTC elimination chart
fig_ctc_elimination = create_unified_income_comparison_graph(state_code, scenario='ctc_elimination')
fig_ctc_elimination = format_fig(fig_ctc_elimination)
fig_ctc_elimination.show()